In [ ]:
# Importing necesary libraries
import pandas as pd
import numpy as np

In [ ]:
# Reading csv and storing as dataframe
df = pd.read_csv("/content/raw_data.csv")
df.head()

,Timestamp,Gender,Area,Whether the student lives in a hostel or commutes from home?,Higher Secondary Education,Higher Secondary Subjects,Higher Secondary Marks Percentage (0-100) (-1 if unsure),NET Marks (0-200) (-1 for ACT),The highest level of education attained by the student’s parents?,Total Family Income,...,Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Your Self-Confidence Level],Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Your Study Consistency Level],Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Participation in Extra-Curricular Activities],Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Emotional Support from Parents/Family/Friends],"Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Sleep Pattern (1: <4hours, 2: 4-6hours, 3: 6-8hours, 4: 8-10hours, 5: 10+hours]","Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Attendance (1: 0-20%, 5: 80-100%)]","Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Access to Resources (Computers, Learning Materials, Mentors etc.)]","Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Understanding of Subject (Programming, Engineering etc.), Before Joining University]",Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Interest in Chosen Degree (e.g interest in CS/EE etc))],1st Semester GPA (First SGPA) (0-4)
0,3/5/2025 13:17:21,Male,Urban,Day Scholar,FSC,Pre Engineering,93,165,Graduation (BS/BA),"250,000 to 350,000",...,2,4,3,5,4,5,5,4,5,3.7
1,3/5/2025 13:17:24,Male,Urban,Hostelite,FSC,Pre Engineering,95,145,Post Graduation (MS/MA),"150,000 to 250,000",...,4,4,4,4,5,4,4,4,4,3.4
2,3/5/2025 13:17:24,Male,Urban,Hostelite,FSC,Computer Science,93,143,Post Graduation (MS/MA),"250,000 to 350,000",...,4,2,3,3,3,5,5,2,3,3.31
3,3/5/2025 13:17:29,Male,Urban,Hostelite,A Levels,Pre Engineering,88,150,Post Graduation (MS/MA),"250,000 to 350,000",...,3,1,3,3,3,4,5,5,5,3.4
4,3/5/2025 13:17:31,Male,Urban,Hostelite,FSC,Pre Engineering,89,143,Post Graduation (MS/MA),Alhamdullillah,...,3,3,5,5,1,5,5,1,5,3.3


In [ ]:
# Dropping rows with empty target variable
df = df[df['1st Semester GPA (First SGPA) (0-4)'].notna()]
df = df[df['1st Semester GPA (First SGPA) (0-4)'] != ""]

# Converting target variable(SGPA) to float datatype
df['1st Semester GPA (First SGPA) (0-4)'] = pd.to_numeric(df['1st Semester GPA (First SGPA) (0-4)'], errors='coerce')
df['1st Semester GPA (First SGPA) (0-4)'].dtypes

dtype('float64')

In [ ]:
# Checking the target variable
df["1st Semester GPA (First SGPA) (0-4)"].isna().sum()

np.int64(0)

In [ ]:
# Handling noisy data

# Removing percentage signs from values
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] = df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].replace('%','',regex = True)

# Convert to numeric, invalid values will be converted to NaN
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] = pd.to_numeric(df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'], errors='coerce')

# Converting fsc total marks to percentage
total_marks = 1100
def percentage_converter(marks):
  if marks > 100: # Convert only if marks are not written as percentage
    return (marks / total_marks) * 100
  return marks #If value is already in percentage then return marks as it is

df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] = df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].apply(percentage_converter)

# Checking cleaned data
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].head(10)

# Changing value at row no 7 as it is wrongly written as 194
df.at[6, 'Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] = 94  # Change the value at row 7 (index 6) to 94

# Checking the change
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].head(10)


,Higher Secondary Marks Percentage (0-100) (-1 if unsure)
0,93.000000
1,95.000000
2,93.000000
3,88.000000
4,89.000000
5,90.636364
6,94.000000
7,92.000000
9,85.000000
10,89.000000


In [ ]:
# Calculate mean (excluding -1 if still present)
perc_mean = df[df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] != -1]['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].mean()

# Replace -1 with the calculated mean
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] = df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].replace(-1, perc_mean)

# Checking the change
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].head(10)

,Higher Secondary Marks Percentage (0-100) (-1 if unsure)
0,93.000000
1,95.000000
2,93.000000
3,88.000000
4,89.000000
5,90.636364
6,94.000000
7,92.000000
9,85.000000
10,89.000000


In [ ]:
#Replacing missing values with mean aswell

# Make sure the column is numeric
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] = pd.to_numeric(df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'], errors='coerce')

# Calculate mean
mean_value = df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].mean()

# Replace only NaN (empty rows) with the mean
df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'] = df['Higher Secondary Marks Percentage (0-100) (-1 if unsure)'].fillna(mean_value)

In [ ]:
# Standardize values: strip whitespace and lowercase
df['Total Family Income'] = df['Total Family Income'].str.strip().str.lower()

# Get mode of cleaned data
mode_value = df['Total Family Income'].mode()[0]

# Replace standardized noisy entries
df['Total Family Income'] = df['Total Family Income'].replace({
    'alhamdullillah': mode_value,
    'alhumdulillah': mode_value,
    'my father earns in dollars': mode_value
})


In [ ]:
# Converting family size to class mark
income_mapping = {
    'Below 25,000': 25000,
    '25,000 to 50,000': 37500,
    '50,000 to 150,000': 100000,
    '150,000 to 250,000': 200000,
    '250,000 to 350,000': 300000,
    '350,000 to 450,000': 400000,
    '450,000 above': 500000
}

# Apply the mapping to the column
df['Family Income'] = df['Total Family Income'].map(income_mapping)

# Check if the conversion was successful
df[['Total Family Income', 'Family Income']].head()



,Total Family Income,Family Income
0,"250,000 to 350,000",300000.0
1,"150,000 to 250,000",200000.0
2,"250,000 to 350,000",300000.0
3,"250,000 to 350,000",300000.0
4,"150,000 to 250,000",200000.0


In [ ]:
# Converting house size to class mark
house_mapping = {
    'less than 5 marla': 3,
    '5 - 9 marla': 7,
    '10 - 19 marla': 15,
    '5 - 10 marla': 7,
    '10 - 20 marla': 15,
    '1 - 2 kanal': 30,
    '2 - 4 kanal': 60,
    '4+ kanal': 90
    }

# Apply the mapping to the column
df['House Marla Size'] = df['House Size'].map(house_mapping)

# Check if the conversion was successful
df[['House Size', 'House Marla Size']].head()


,House Size,House Marla Size
0,10 - 20 marla,15
1,1 - 2 kanal,30
2,5 - 10 marla,7
3,5 - 10 marla,7
4,5 - 10 marla,7


In [ ]:
# Replacing net marks missing values with mean

# Data formatted as marks/200
df['NET Marks (0-200) (-1 for ACT)'] = df['NET Marks (0-200) (-1 for ACT)'].replace('/200','',regex = True)

# Data having invalid strings
df['NET Marks (0-200) (-1 for ACT)'] = pd.to_numeric(df['NET Marks (0-200) (-1 for ACT)'], errors='coerce')

# Replace -1 and invalid strings (already handled by 'coerce') with NaN
df['NET Marks (0-200) (-1 for ACT)'].replace(-1, np.nan, inplace=True)

# Now calculate the mean (excluding NaNs)
net_mean = df['NET Marks (0-200) (-1 for ACT)'].mean()

# Replace NaNs with the calculated mean
df['NET Marks (0-200) (-1 for ACT)'].fillna(net_mean, inplace=True)
df.head(20)


<ipython-input-33-406a2b8ccb8d>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['NET Marks (0-200) (-1 for ACT)'].replace(-1, np.nan, inplace=True)
<ipython-input-33-406a2b8ccb8d>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

,Gender,Area,Whether the student lives in a hostel or commutes from home?,Higher Secondary Education,Higher Secondary Subjects,Higher Secondary Marks Percentage (0-100) (-1 if unsure),NET Marks (0-200) (-1 for ACT),The highest level of education attained by the student’s parents?,Total Family Income,House Size,...,Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Emotional Support from Parents/Family/Friends],"Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Sleep Pattern (1: <4hours, 2: 4-6hours, 3: 6-8hours, 4: 8-10hours, 5: 10+hours]","Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Attendance (1: 0-20%, 5: 80-100%)]","Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Access to Resources (Computers, Learning Materials, Mentors etc.)]","Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Understanding of Subject (Programming, Engineering etc.), Before Joining University]",Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Interest in Chosen Degree (e.g interest in CS/EE etc))],1st Semester GPA (First SGPA) (0-4),Family Income,House Marla Size,Parents Education
0,Male,Urban,Day Scholar,FSC,Pre Engineering,93.000000,165.00000,Graduation (BS/BA),"250,000 to 350,000",10 - 20 marla,...,5,4,5,5,4,5,3.7,300000.0,15,4
1,Male,Urban,Hostelite,FSC,Pre Engineering,95.000000,145.00000,Post Graduation (MS/MA),"150,000 to 250,000",1 - 2 kanal,...,4,5,4,4,4,4,3.4,200000.0,30,5
2,Male,Urban,Hostelite,FSC,Computer Science,93.000000,143.00000,Post Graduation (MS/MA),"250,000 to 350,000",5 - 10 marla,...,3,3,5,5,2,3,3.3,300000.0,7,5
3,Male,Urban,Hostelite,A Levels,Pre Engineering,88.000000,150.00000,Post Graduation (MS/MA),"250,000 to 350,000",5 - 10 marla,...,3,3,4,5,5,5,3.4,300000.0,7,5
4,Male,Urban,Hostelite,FSC,Pre Engineering,89.000000,143.00000,Post Graduation (MS/MA),"150,000 to 250,000",5 - 10 marla,...,5,1,5,5,1,5,3.3,200000.0,7,5
5,Female,Urban,Hostelite,FSC,Pre Medicine,90.636364,144.00000,Graduation (BS/BA),"50,000 to 150,000",5 - 10 marla,...,4,3,5,5,5,4,2.7,100000.0,7,4
6,Male,Urban,Day Scholar,A Levels,Pre Engineering,94.000000,145.00000,Graduation (BS/BA),"250,000 to 350,000",5 - 10 marla,...,5,3,4,5,3,4,3.8,300000.0,7,4
7,Female,Urban,Hostelite,FSC,Pre Engineering,92.000000,140.00000,Graduation (BS/BA),"below 50,000",5 - 10 marla,...,4,4,5,4,3,3,3.1,NaN,7,4
9,Male,Urban,Hostelite,FSC,Pre Engineering,85.000000,150.00000,Graduation (BS/BA),"150,000 to 250,000",10 - 20 marla,...,4,3,4,3,2,2,2.8,200000.0,15,4
10,Male,Urban,Hostelite,FSC,Pre Engineering,89.000000,143.00000,Higher Secondary (12th grade),"50,000 to 150,000",5 - 10 marla,...,4,4,4,2,2,2,3.3,100000.0,7,3


In [ ]:
df['NET Marks (0-200) (-1 for ACT)'].isna().sum()    # Gives the total count of NaN values per column


np.int64(0)

In [ ]:
# Converting parents education to ordinal data
# Define the ordinal mapping for education levels
education_mapping = {
    'Primary (5th grade)': 1,
    'Secondary (10th grade)': 2,
    'Higher Secondary (12th grade)': 3,
    'Graduation (BS/BA)': 4,
    'Post Graduation (MS/MA)': 5,
    'PhD / ScD': 6
}

# Apply the mapping to the column
df['Parents Education'] = df['The highest level of education attained by the student’s parents?'].map(education_mapping)

# Check if the conversion was successful
df[['The highest level of education attained by the student’s parents?', 'Parents Education']].head()



,The highest level of education attained by the student’s parents?,Parents Education
0,Graduation (BS/BA),4
1,Post Graduation (MS/MA),5
2,Post Graduation (MS/MA),5
3,Post Graduation (MS/MA),5
4,Post Graduation (MS/MA),5


In [ ]:
# Round the 'GPA' column to 1 digit after decimal
df['1st Semester GPA (First SGPA) (0-4)'] = df['1st Semester GPA (First SGPA) (0-4)'].round(1)

# Check the result
df[['1st Semester GPA (First SGPA) (0-4)']].head()

,1st Semester GPA (First SGPA) (0-4)
0,3.7
1,3.4
2,3.3
3,3.4
4,3.3


In [ ]:
# Drop the redundant columns
df.drop('Total Family Income', axis=1, inplace=True)
df.drop('House Size', axis=1, inplace=True)

# Check the DataFrame to confirm it's dropped
df.head()

,Gender,Area,Whether the student lives in a hostel or commutes from home?,Higher Secondary Education,Higher Secondary Subjects,Higher Secondary Marks Percentage (0-100) (-1 if unsure),NET Marks (0-200) (-1 for ACT),The highest level of education attained by the student’s parents?,Family Size (Number of People living at your house),Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Your Stress Level],...,Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Emotional Support from Parents/Family/Friends],"Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Sleep Pattern (1: <4hours, 2: 4-6hours, 3: 6-8hours, 4: 8-10hours, 5: 10+hours]","Please answer to the best of your judgement / self awareness. (1: lowest - 5: highest) [Attendance (1: 0-20%, 5: 80-100%)]","Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Access to Resources (Computers, Learning Materials, Mentors etc.)]","Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Understanding of Subject (Programming, Engineering etc.), Before Joining University]",Please answer regarding your life BEFORE JOINING UNIVERSITY. (1:lowest - 5:highest) [Interest in Chosen Degree (e.g interest in CS/EE etc))],1st Semester GPA (First SGPA) (0-4),Family Income,House Marla Size,Parents Education
0,Male,Urban,Day Scholar,FSC,Pre Engineering,93.0,165.0,Graduation (BS/BA),2,2,...,5,4,5,5,4,5,3.7,300000.0,15,4
1,Male,Urban,Hostelite,FSC,Pre Engineering,95.0,145.0,Post Graduation (MS/MA),5,3,...,4,5,4,4,4,4,3.4,200000.0,30,5
2,Male,Urban,Hostelite,FSC,Computer Science,93.0,143.0,Post Graduation (MS/MA),5,3,...,3,3,5,5,2,3,3.3,300000.0,7,5
3,Male,Urban,Hostelite,A Levels,Pre Engineering,88.0,150.0,Post Graduation (MS/MA),4,2,...,3,3,4,5,5,5,3.4,300000.0,7,5
4,Male,Urban,Hostelite,FSC,Pre Engineering,89.0,143.0,Post Graduation (MS/MA),10,1,...,5,1,5,5,1,5,3.3,200000.0,7,5


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('cleaned_dataset.csv', index=False)